In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras import Model

from json import loads

%matplotlib inline

In [2]:
(X,Y),(x,y) = tf.keras.datasets.mnist.load_data()

X = X.reshape(-1,784) / 255
x = x.reshape(-1,784) / 255

Y = tf.keras.utils.to_categorical(Y)
y = tf.keras.utils.to_categorical(y)

In [51]:
inp = Input(shape=(784,),name="input")

d1 = Dense(128,activation="relu",name="dense_1")(inp)
d2a = Dense(32,activation="relu",name="dense_2a")(d1)
d2b = Dense(32,activation="relu",name="dense_2b")(d1)
c = concatenate([d2a,d2b],name='add')
d3 = Dense(32,activation="relu",name="dense_3")(c)

out = Dense(10,activation="softmax",name="out")(d3)

model = Model([inp],[out])

In [52]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          100480      input[0][0]                      
__________________________________________________________________________________________________
dense_2a (Dense)                (None, 32)           4128        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_2b (Dense)                (None, 32)           4128        dense_1[0][0]                    
____________________________________________________________________________________________

In [53]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [54]:
model.fit(X,Y,epochs=1,batch_size=32,validation_data=(x,y))

1875/1875 [==============================] - 4s 2ms/step - loss: 0.4596 - accuracy: 0.8573 - val_loss: 0.1243 - val_accuracy: 0.9634


In [55]:
model_config = loads(model.to_json())

In [145]:
config_names = ['units']

network = {}

for layer in model_config['config']['layers']:
    la = {}
    la['level'] = 0
    la['inbound_nodes'] = [l[0] for l in layer['inbound_nodes'][0]] if layer['inbound_nodes'] else []
    la['outbound_nodes'] = []
    la['config'] = { key:layer['config'][key] for key in config_names if key in layer['config']}
    network[layer['config']['name']] = la
    
for l in network:
    for n in network[l]['inbound_nodes']:
        network[n]['outbound_nodes'].append(l)
        
for i,layer in enumerate(model.layers):
    network[layer.name]['level'] = i

In [146]:
network['dense_3']

{'level': 5,
 'inbound_nodes': ['add'],
 'outbound_nodes': ['out'],
 'config': {'units': 32}}

In [149]:
network

{'input': {'level': 0,
  'inbound_nodes': [],
  'outbound_nodes': ['dense_1'],
  'config': {}},
 'dense_1': {'level': 1,
  'inbound_nodes': ['input'],
  'outbound_nodes': ['dense_2a', 'dense_2b'],
  'config': {'units': 128}},
 'dense_2a': {'level': 2,
  'inbound_nodes': ['dense_1'],
  'outbound_nodes': ['add'],
  'config': {'units': 32}},
 'dense_2b': {'level': 3,
  'inbound_nodes': ['dense_1'],
  'outbound_nodes': ['add'],
  'config': {'units': 32}},
 'add': {'level': 4,
  'inbound_nodes': ['dense_2a', 'dense_2b'],
  'outbound_nodes': ['dense_3'],
  'config': {}},
 'dense_3': {'level': 5,
  'inbound_nodes': ['add'],
  'outbound_nodes': ['out'],
  'config': {'units': 32}},
 'out': {'level': 6,
  'inbound_nodes': ['dense_3'],
  'outbound_nodes': [],
  'config': {'units': 10}}}

In [15]:
!mkdir "../save_models/MNIST_DENSE/"

A subdirectory or file ../save_models/MNIST_DENSE/ already exists.


In [16]:
open("../save_models/MNIST_DENSE/model.json","w+").write(model.to_json())
model.save_weights("../save_models/MNIST_DENSE/model")